# Génération d'un fichier csv 
### Ce script doit générer un fichier csv contenant des données propres, qui seront par la suite retravaillées pour faire de la prédictions d'ingrédients ou de catégorie dans l'objectif de prédire l'écoscore des produits. 

In [40]:
import tensorflow as tf

def setup_gpu():
    physical_devices = tf.config.list_physical_devices('GPU')
    if physical_devices:
        try:
            tf.config.experimental.set_memory_growth(physical_devices[0], True)
            print("config augmentation allocation mémoire gpu activée")
        except RuntimeError as e:
            print(e)

setup_gpu()

config augmentation allocation mémoire gpu activée


In [41]:
import keras
import numpy as np
import math
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sea
import sklearn
import scipy as sc
import nltk as nltk
import statsmodels as statsmodels
import os

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [42]:
print("Version de TensorFlow :", tf.__version__)

Version de TensorFlow : 2.10.1


In [43]:
print("Version de Keras :", keras.__version__)

Version de Keras : 2.10.0


## Panneau de configuration

In [44]:
project_path = "C:\\Users\\charl\\Documents\\workspace\\green_ia\\cch\\"
file_nbr = '01' # numéro d'identification des csv à générer 
openfoodfact_csv_version = '01'

In [45]:
# import du csv de données openfoodfacts
openfoodfacts_csv = project_path + f"data_global\\openfoodfacts_{openfoodfact_csv_version}.csv"
df_imported = pd.read_csv(openfoodfacts_csv)

C:\Users\charl\AppData\Local\Temp\ipykernel_12936\2487915700.py:3: DtypeWarning: Columns (0,11,12,14,15,16,17,23,24,25,26,27,31,32,33,34,35,36,37,44,46,47,48,50,52,56,67,72) have mixed types. Specify dtype option on import or set low_memory=False.
  df_imported = pd.read_csv(openfoodfacts_csv)


In [185]:
# cellule à rafraichir pour éviter de charger pendant 1h !
df = df_imported

# Analyse des données 

In [186]:
# affiche le nom de toutes les colonnes du df 
column_list = list(df.columns)
print(f"nbr col: {len(column_list)}, liste: {column_list}")

nbr col: 206, liste: ['code', 'url', 'creator', 'created_t', 'created_datetime', 'last_modified_t', 'last_modified_datetime', 'last_modified_by', 'last_updated_t', 'last_updated_datetime', 'product_name', 'abbreviated_product_name', 'generic_name', 'quantity', 'packaging', 'packaging_tags', 'packaging_en', 'packaging_text', 'brands', 'brands_tags', 'categories', 'categories_tags', 'categories_en', 'origins', 'origins_tags', 'origins_en', 'manufacturing_places', 'manufacturing_places_tags', 'labels', 'labels_tags', 'labels_en', 'emb_codes', 'emb_codes_tags', 'first_packaging_code_geo', 'cities', 'cities_tags', 'purchase_places', 'stores', 'countries', 'countries_tags', 'countries_en', 'ingredients_text', 'ingredients_tags', 'ingredients_analysis_tags', 'allergens', 'allergens_en', 'traces', 'traces_tags', 'traces_en', 'serving_size', 'serving_quantity', 'no_nutrition_data', 'additives_n', 'additives', 'additives_tags', 'additives_en', 'nutriscore_score', 'nutriscore_grade', 'nova_group'

In [187]:
# afficher % de nan / colonne:
total_rows = df.shape[0]
nan_counts = df.isna().sum()
nan_percentage = (nan_counts / total_rows) * 100
nan_df = pd.DataFrame({'column_name': nan_percentage.index, 'percentage_nan': nan_percentage.values})

nan_percent_range = [(0, 10), (10, 20), (20, 30), (30, 40), (40, 50), (50, 60), (60, 70), (70, 80), (80, 90), (90, 100)]
grouped = nan_df.groupby(pd.cut(nan_df['percentage_nan'], bins=[tranche[0] for tranche in nan_percent_range + [(100,)]]))

for tranche, group in grouped:
    print(f"range {tranche}:")
    print(group['column_name'].tolist())
    print()

range (0, 10]:
['creator', 'last_modified_by', 'last_updated_t', 'last_updated_datetime', 'product_name', 'countries', 'countries_tags', 'countries_en', 'nutriscore_grade', 'pnns_groups_1', 'pnns_groups_2', 'states', 'states_tags', 'states_en', 'ecoscore_grade', 'completeness']

range (10, 20]:
['last_image_t', 'last_image_datetime', 'image_url', 'image_small_url']

range (20, 30]:
['energy-kcal_100g', 'energy_100g', 'fat_100g', 'saturated-fat_100g', 'carbohydrates_100g', 'sugars_100g', 'proteins_100g']

range (30, 40]:
['salt_100g', 'sodium_100g']

range (40, 50]:
['brands', 'brands_tags', 'image_nutrition_url', 'image_nutrition_small_url']

range (50, 60]:
['categories', 'categories_tags', 'categories_en', 'main_category', 'main_category_en']

range (60, 70]:
['quantity', 'ingredients_analysis_tags', 'nutriscore_score', 'food_groups', 'food_groups_tags', 'food_groups_en', 'nutrient_levels_tags', 'product_quantity', 'unique_scans_n', 'popularity_tags', 'fiber_100g', 'nutrition-score-f

C:\Users\charl\AppData\Local\Temp\ipykernel_12936\3943661832.py:8: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped = nan_df.groupby(pd.cut(nan_df['percentage_nan'], bins=[tranche[0] for tranche in nan_percent_range + [(100,)]]))


# Traitement des données 

In [188]:
# récupérer les colonnes intéressantes 
col_classi = [
    'pnns_groups_2',
    'energy-kcal_100g',
    'fat_100g',
    'saturated-fat_100g',
    'carbohydrates_100g',
    'sugars_100g',
    'proteins_100g',
    'salt_100g',
    'sodium_100g',
    'brands_tags',
]
df = df[col_classi] 
df.tail(3)

,pnns_groups_2,energy-kcal_100g,fat_100g,saturated-fat_100g,carbohydrates_100g,sugars_100g,proteins_100g,salt_100g,sodium_100g,brands_tags
3236623,unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,mtr
3236624,Dairy desserts,24.0,28.0,13.0,70.0,49.0,2.0,3.0,1.2,NaN
3236625,unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,edeka


In [189]:
# renommer les colonnes 
rename_col = {
    'energy-kcal_100g': 'kcal', 
    'fat_100g': 'fat',
    'saturated-fat_100g': 'sat_fat',
    'carbohydrates_100g': 'carbohyd',
    'sugars_100g': 'sugar',
    'proteins_100g': 'prot',
    'salt_100g': 'salt',
    'sodium_100g': 'sodium',
    'brands_tags': 'brand',
    'pnns_groups_2': 'pnns2'
}
df = df.rename(columns=rename_col)
df.tail(3)

,pnns2,kcal,fat,sat_fat,carbohyd,sugar,prot,salt,sodium,brand
3236623,unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,mtr
3236624,Dairy desserts,24.0,28.0,13.0,70.0,49.0,2.0,3.0,1.2,NaN
3236625,unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,edeka


In [190]:
# remplacer tous les NaN et unkown par null 
df.replace(['unknown', np.nan], ['none', 'none'], inplace=True)
df.tail(3)

,pnns2,kcal,fat,sat_fat,carbohyd,sugar,prot,salt,sodium,brand
3236623,none,none,none,none,none,none,none,none,none,mtr
3236624,Dairy desserts,24.0,28.0,13.0,70.0,49.0,2.0,3.0,1.2,none
3236625,none,none,none,none,none,none,none,none,none,edeka


In [191]:
# mélange des lignes aléatoirement 
df = df.sample(frac=1).reset_index(drop=True)
df.tail(3)

,pnns2,kcal,fat,sat_fat,carbohyd,sugar,prot,salt,sodium,brand
3236623,none,none,none,none,none,none,none,none,none,none
3236624,none,530.0,31.8,7.07,49.47,0.0,7.07,0.84,0.336,utz
3236625,Salty and fatty products,none,none,none,none,none,none,none,none,none


In [196]:
# supprime les lignes où ni pnss2 et ni brand ont du contenu 
df = df[~((df['pnns2'] == 'none') & (df['brand'] == 'none'))]
df.tail(3)

,pnns2,kcal,fat,sat_fat,carbohyd,sugar,prot,salt,sodium,brand
3236622,none,170.0,11.0,2.5,17.0,11.0,0.0,0.1625,0.065,lotus
3236624,none,530.0,31.8,7.07,49.47,0.0,7.07,0.84,0.336,utz
3236625,Salty and fatty products,none,none,none,none,none,none,none,none,none


In [197]:
# si cette colonne indique un ingrédient est vide et pnns2 est vide également:
    # on supprime la ligne 
# on supprime la colonne 

# création colonne vaut 0 si manque aucun ingrédient, 1 si en manque 1 sur une ligne 
col_to_check = ['kcal', 'fat', 'sat_fat', 'carbohyd', 'sugar', 'prot', 'salt', 'sodium']
mask = df[col_to_check].apply(lambda x: x.eq('none')).any(axis=1)
df['ingr_miss_val'] = np.where(mask, 1, 0)
df.tail(30)

,pnns2,kcal,fat,sat_fat,carbohyd,sugar,prot,salt,sodium,brand,missing_data
3236583,Cheese,419.0,38.0,27.0,0.0,0.0,19.0,1.9,0.76,ametller-origen,0
3236584,Dressings and sauces,none,none,none,none,none,none,none,none,none,1
3236585,Sweets,554.0,35.0,13.0,45.0,45.0,11.0,0.15,0.06,malakoff,0
3236586,none,464.0,21.43,17.86,67.86,35.71,3.57,0.2675,0.107,4-him-food-group-llc,0
3236587,One-dish meals,219.0,6.5,3.6,32.0,2.0,10.0,0.9,0.36,ametller-origen,0
3236588,Fruits,64.0,0.0,0.0,17.14,15.0,0.71,0.0,0.0,simply-balanced,0
3236591,none,17.0,0.0,0.0,4.2,4.2,0.0,0.03,0.012,vitamin-well,0
3236594,Fats,594.0,53.12,12.5,21.88,9.38,18.75,0.43,0.172,maranatha,0
3236595,Pastries,365.0,13.0,8.0,55.0,25.0,3.0,0.37,0.148,chiostro-di-saronno,0
3236596,Cheese,214.0,16.07,8.93,3.57,3.57,10.71,0.715,0.286,los-altos-food-products-inc,0


In [177]:
# normalisation des données numériques 
#col_to_norm = ['kcal', 'fat', 'sat_fat', 'carbohyd', 'sugar', 'prot', 'salt', 'sodium']
#scaler = MinMaxScaler()
#df[col_to_norm] = scaler.fit_transform(df[col_to_norm])
#df.tail(3)

In [178]:
# arrondir toutes les valeurs numériques, ne garder aucun chiffre après la virgule
#col_to_round = ['kcal', 'fat', 'sat_fat', 'carbohyd', 'sugar', 'prot', 'salt', 'sodium']
#df[col_to_round] = df[col_to_round].round()
#df[col_to_round] = df[col_to_round].astype(int)
#df.tail(30)